In [1]:
import os
from io import StringIO
import tabula

# analysis
import numpy as np
import pandas as pd

# api
import json
import requests
import prettytable

# Get data

**What data do we need?**
- GDP by states by industry: https://www.bea.gov/system/files/2020-01/qgdpstate0120_2.pdf
- Employmnet by states by industry:
    - In order to get the data, we probably need to get the employment data by industry from each states' website.
        - NY: https://labor.ny.gov/stats/2016-2026-Statewide-and-Regional-Long-Term-Industry-Projections.xlsx
        - NJ: https://www.nj.gov/labor/lpa/employ/indoccpj/20162026%20Industry%20Employment%20Projections%20Reports.xlsx
        - ...so the 2016-2026 employment projection is avaliable on each states websites
        
- Labor Productivity by states by industry:
    - Private nonfarm productivity and costs by state and region: https://www.bls.gov/lpc/lpc-by-state-and-region.xlsx
    - https://www.bls.gov/opub/mlr/2019/article/bls-publishes-experimental-state-level-labor-productivity-measures.htm
        

## 1.0 GDP by states by industry
**API:** https://www.bls.gov/developers/api_signature_v2.htm

#### Try API

In [151]:
# my_bls_api = "ccdcbc9910a744a5be046d1aa70ac25c"

In [152]:
# headers = {'Content-type': 'application/json'}
# data = json.dumps({"seriesid": ['IPUAN1111__L010','IPUAN1111__L011'],"startyear":"2000", "endyear":"2019"})
# p = requests.post('https://api.bls.gov/ccdcbc9910a744a5be046d1aa70ac25c/v2/timeseries/data/', data=data, headers=headers)
# json_data = json.loads(p.text)
# for series in json_data['Results']['series']:
#     x=prettytable.PrettyTable(["series_id","year","period","value","footnotes"])
#     seriesId = series['seriesID']
#     for item in series['data']:
#         year = item['year']
#         period = item['period']
#         value = item['value']
#         footnotes=""
#         for footnote in item['footnotes']:
#             if footnote:
#                 footnotes = footnotes + footnote['text'] + ','
# #         if 'M01' <= period <= 'M12':
#         x.add_row([seriesId,year,period,value,footnotes[0:-1]])
#     output = open(seriesId + '.txt','w')
#     output.write (x.get_string())
#     output.close()

#### Reading from PDF


#### 1. Current-Dollar Gross Domestic Product (GDP) by State and Region, 2018:Q1-2019:Q3

In [157]:
gdp_df = tabula.read_pdf(os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf"), 
                         pages = 7)

In [158]:
gdp_df = gdp_df[0]

In [159]:
gdp_df = gdp_df.iloc[3:]

In [160]:
gdp_df = gdp_df.iloc[:,[0,1,2,3,4]]

In [161]:
gdp_df = pd.concat([gdp_df,gdp_df["Millions of dollars"].str.split(" ",expand = True)],axis=1)

In [162]:
gdp_df = gdp_df.drop(columns=["Millions of dollars","Unnamed: 3",5,6,7,8])

In [163]:
gdp_df.columns = ["areas","2018-1","2018-2","2018-3","2018-4","2019-1","2019-2","2019-3"]

In [164]:
gdp_df = gdp_df.reset_index(drop=True)

In [165]:
gdp_df.head()

,areas,2018-1,2018-2,2018-3,2018-4,2019-1,2019-2,2019-3
0,United States,"20,163,159","20,510,177","20,749,752","20,897,804","21,098,827","21,340,267","21,542,540"
1,New England,"1,070,863","1,084,947","1,096,174","1,101,531","1,118,703","1,130,084","1,142,586"
2,Connecticut,"272,854","273,884","278,295","277,874","281,659","284,357","287,560"
3,Maine,"63,698","64,746","65,434","65,545","66,590","67,138","67,905"
4,Massachusetts,"558,472","568,220","573,506","577,754","586,347","592,588","599,092"


#### 2. Contributions to Percent Change in Real Gross Domestic Product (GDP) by State and Region, 2019:Q2-2019:Q3--Table Ends

### Part 1

In [392]:
gdp_by_state_by_inds_p1 = tabula.read_pdf(os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf"), 
                         pages = 5)

In [393]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1[0]

In [394]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.dropna(axis=0, how='all').dropna(axis=1, how='all')

In [395]:
gdp_by_state_by_inds_p1.head(6)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Seasonally adjusted at annual rates,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,NaN,NaN,NaN,NaN,NaN,NaN,Percentage points,NaN,NaN,NaN
1,NaN,Percent change,NaN,"Mining,",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,in real GDP by,"Agriculture,","quarrying, and",NaN,NaN,Durable goods Nondurable goods,NaN,Transportation and,Finance and
3,NaN,state,"forestry, fishing insuranceand hunting",Informationoil and gas,Utilities,Construction,Wholesale trade manufacturing manufacturing,Retail trade,warehousing,NaN
4,NaN,NaN,NaN,extraction,NaN,NaN,NaN,NaN,NaN,NaN
5,United States,2.1,0.06,0.11,-0.26,-0.09,0.01 0.46 0.18,0.43,-0.03,0.29 -0.42


In [396]:
gdp_by_state_by_inds_p1.iloc[3]

Unnamed: 0                                                                     NaN
Unnamed: 1                                                                   state
Unnamed: 2                                  forestry, fishing insuranceand hunting
Unnamed: 3                                                  Informationoil and gas
Unnamed: 4                                                               Utilities
Unnamed: 5                                                            Construction
Seasonally adjusted at annual rates    Wholesale trade manufacturing manufacturing
Unnamed: 6                                                            Retail trade
Unnamed: 7                                                             warehousing
Unnamed: 8                                                                     NaN
Name: 3, dtype: object

In [397]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.loc[5:]

In [398]:
gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Seasonally adjusted at annual rates'].str.split(" ", expand=True)], axis=1)

In [399]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Seasonally adjusted at annual rates'])

In [404]:
gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Unnamed: 8'].str.split(" ", expand=True)], axis=1)

In [406]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Unnamed: 8'])

In [407]:
gdp_by_state_by_inds_p1.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,0,1,2,0,1
5,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,0.18,0.29,-0.42
6,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,0.19,0.34,-0.47
7,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,0.30,0.36,-0.55
8,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,0.24,0.17,-0.43
9,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,0.12,0.40,-0.43


In [408]:
cols = ["areas", "states_overall","Agriculture,forestry, fishing insuranceand hunting", 
        "Mining,quarrying, and oil and gas extraction","Utilities","Construction", 
        "Retail trade","Transportation and warehousing","Durable goods manufacturing",
        "Nondurable goods manufacturing", "Wholesale trade", "Information", 
        "Finance and insurance"]

In [409]:
gdp_by_state_by_inds_p1.columns = cols

In [420]:
gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.reset_index(drop=True)

### Part 2

In [345]:
gdp_by_state_by_inds_p2 = tabula.read_pdf(os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf"), 
                         pages = 6)

In [346]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[0]

In [347]:
gdp_by_state_by_inds_p2.iloc[:5,:5]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,NaN,NaN,NaN,NaN,NaN
1,NaN,Other services (except Real estate and governm...,Management of companies and enterprises,NaN,NaN
2,United States,0.11 0.42,0.16,NaN,NaN
3,New England,0.09 0.56,0.20,NaN,NaN
4,Connecticut,0.03 0.37,0.19,NaN,NaN


In [348]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[2:]

In [351]:
gdp_by_state_by_inds_p2[["Real estate and rental and leasing",
                      "Professional, scientific, and technical services"]] = gdp_by_state_by_inds_p2["Unnamed: 1"].str.split(" ",expand = True)

In [352]:
gdp_by_state_by_inds_p2[["Educational services",
                      "Health care and social assistance"]] = gdp_by_state_by_inds_p2[
                                                            "Seasonally adjusted at annual rates"
                                                                                  ].str.split(" ", expand = True)

In [353]:
gdp_by_state_by_inds_p2[["Other services (except government and government enterprises)",
                      "Government and government enterprises"]] = gdp_by_state_by_inds_p2["Unnamed: 14"].str.split(" ", expand = True)

In [354]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.drop(columns= ["Unnamed: 1","Unnamed: 14","Seasonally adjusted at annual rates"])

In [355]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.dropna(axis=1)

In [356]:
gdp_by_state_by_inds_p2.iloc[:5,:5]

,Unnamed: 0,Unnamed: 2,Unnamed: 5,Unnamed: 9,Unnamed: 11
2,United States,0.16,0.10,0.02,0.17
3,New England,0.20,0.12,0.04,0.17
4,Connecticut,0.19,0.11,0.04,0.11
5,Maine,0.27,0.15,0.05,0.33
6,Massachusetts,0.20,0.10,0.03,0.16


In [357]:
name_dic = {"Unnamed: 0":"areas",
            "Unnamed: 2":"Management of companies and enterprises",
            "Unnamed: 5":"Administrative and support and waste management and remediation services",
            "Unnamed: 9":"Arts, entertainment, and recreation", 
            "Unnamed: 11":"Accomodation and food services"}

In [358]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.rename(columns=name_dic)

In [418]:
gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.reset_index(drop=True)

In [419]:
gdp_by_state_by_inds_p2.dtypes

areas                                                                        object
management_of_companies_and_enterprises                                      object
administrative_and_support_and_waste_management_and_remediation_services     object
arts_entertainment_and_recreation                                           float64
accomodation_and_food_services                                               object
educational_services                                                         object
health_care_and_social_assistance                                            object
real_estate_and_rental_and_leasing                                           object
professional_scientific_and_technical_services                               object
other_services_except_government_and_government_enterprises_                 object
government_and_government_enterprises                                        object
dtype: object

### Join part one and part two

In [421]:
gdp_by_state_by_inds = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p2], axis = 1,)

In [422]:
gdp_by_state_by_inds.columns = gdp_by_state_by_inds.columns.str.replace(
                                                "\)|\(|,| ","_").str.replace("__","_").str.lower()

In [428]:
gdp_by_state_by_inds = gdp_by_state_by_inds.iloc[:,~gdp_by_state_by_inds.columns.duplicated()]

In [429]:
gdp_by_state_by_inds.head()

,areas,states_overall,agriculture_forestry_fishing_insuranceand_hunting,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,retail_trade,transportation_and_warehousing,durable_goods_manufacturing,nondurable_goods_manufacturing,...,management_of_companies_and_enterprises,administrative_and_support_and_waste_management_and_remediation_services,arts_entertainment_and_recreation,accomodation_and_food_services,educational_services,health_care_and_social_assistance,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,other_services_except_government_and_government_enterprises_,government_and_government_enterprises
0,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,...,0.16,0.10,0.02,0.17,0.08,0.19,0.11,0.42,0.05,0.01
1,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,...,0.20,0.12,0.04,0.17,0.23,0.21,0.09,0.56,0.05,-0.01
2,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,...,0.19,0.11,0.04,0.11,0.27,0.17,0.03,0.37,0.07,-0.04
3,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,...,0.27,0.15,0.05,0.33,0.06,0.15,0.07,0.26,0.05,-0.10
4,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,...,0.20,0.10,0.03,0.16,0.24,0.22,0.13,0.72,0.05,0.00


In [331]:
def get_gdp_by_states(data_dir):
    gdp_df = tabula.read_pdf(data_dir, pages = 7)
    gdp_df = gdp_df[0].iloc[3:,[0,1,2,3,4]]
    gdp_df = pd.concat([gdp_df,gdp_df["Millions of dollars"].str.split(" ",expand = True)],axis=1)
    gdp_df = gdp_df.drop(columns=["Millions of dollars","Unnamed: 3",5,6,7,8])
    gdp_df.columns = ["areas","2018-1","2018-2","2018-3","2018-4","2019-1","2019-2","2019-3"]
    gdp_df = gdp_df.reset_index(drop=True)
    
    return gdp_df

In [332]:
data_dir = os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf")

In [336]:
get_gdp_by_states(data_dir=data_dir).head()

,areas,2018-1,2018-2,2018-3,2018-4,2019-1,2019-2,2019-3
0,United States,"20,163,159","20,510,177","20,749,752","20,897,804","21,098,827","21,340,267","21,542,540"
1,New England,"1,070,863","1,084,947","1,096,174","1,101,531","1,118,703","1,130,084","1,142,586"
2,Connecticut,"272,854","273,884","278,295","277,874","281,659","284,357","287,560"
3,Maine,"63,698","64,746","65,434","65,545","66,590","67,138","67,905"
4,Massachusetts,"558,472","568,220","573,506","577,754","586,347","592,588","599,092"


In [430]:
def get_gdp_pct_by_states_by_inds(data_dir):
    
    # part 1
    gdp_by_state_by_inds_p1 = tabula.read_pdf(data_dir,pages = 5)
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1[0]
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.dropna(axis=0, how='all').dropna(axis=1, how='all')
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.loc[5:]
    gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Seasonally adjusted at annual rates'].str.split(" ", expand=True)], axis=1)
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Seasonally adjusted at annual rates'])
    gdp_by_state_by_inds_p1 = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p1[
                                            'Unnamed: 8'].str.split(" ", expand=True)], axis=1)
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.drop(columns=['Unnamed: 8'])
    cols = ["areas", "states_overall","Agriculture,forestry, fishing insuranceand hunting", 
            "Mining,quarrying, and oil and gas extraction","Utilities","Construction", 
            "Retail trade","Transportation and warehousing","Durable goods manufacturing",
            "Nondurable goods manufacturing", "Wholesale trade", "Information", 
            "Finance and insurance"]
    gdp_by_state_by_inds_p1.columns = cols
    gdp_by_state_by_inds_p1 = gdp_by_state_by_inds_p1.reset_index(drop=True)

    #part 2
    gdp_by_state_by_inds_p2 = tabula.read_pdf(data_dir, pages = 6)

    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[0]
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2[2:]

    gdp_by_state_by_inds_p2[["Real estate and rental and leasing",
                      "Professional, scientific, and technical services"]] = gdp_by_state_by_inds_p2["Unnamed: 1"].str.split(" ",expand = True)

    gdp_by_state_by_inds_p2[["Educational services",
                      "Health care and social assistance"]] = gdp_by_state_by_inds_p2[
                                                            "Seasonally adjusted at annual rates"
                                                                                  ].str.split(" ", expand = True)
    gdp_by_state_by_inds_p2[["Other services (except government and government enterprises)",
                      "Government and government enterprises"]] = gdp_by_state_by_inds_p2["Unnamed: 14"].str.split(" ", expand = True)
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.drop(
                        columns= ["Unnamed: 1","Unnamed: 14","Seasonally adjusted at annual rates"])
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.dropna(axis=1)
    name_dic = {"Unnamed: 0":"areas",
                "Unnamed: 2":"Management of companies and enterprises",
                "Unnamed: 5":"Administrative and support and waste management and remediation services",
                "Unnamed: 9":"Arts, entertainment, and recreation", 
                "Unnamed: 11":"Accomodation and food services"}
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.rename(columns=name_dic)
    gdp_by_state_by_inds_p2 = gdp_by_state_by_inds_p2.reset_index(drop=True)
    # JOIN
    gdp_by_state_by_inds = pd.concat([gdp_by_state_by_inds_p1,gdp_by_state_by_inds_p2], axis = 1,)
    gdp_by_state_by_inds.columns = gdp_by_state_by_inds.columns.str.replace(
                                                "\)|\(|,| ","_").str.replace("__","_").str.lower()
    gdp_by_state_by_inds = gdp_by_state_by_inds.iloc[:,~gdp_by_state_by_inds.columns.duplicated()]
    return gdp_by_state_by_inds

In [431]:
data_dir = os.path.join(os.getcwd(),"data/qgdpstate0120_2.pdf")              

In [432]:
get_gdp_pct_by_states_by_inds(data_dir).head()

,areas,states_overall,agriculture_forestry_fishing_insuranceand_hunting,mining_quarrying_and_oil_and_gas_extraction,utilities,construction,retail_trade,transportation_and_warehousing,durable_goods_manufacturing,nondurable_goods_manufacturing,...,management_of_companies_and_enterprises,administrative_and_support_and_waste_management_and_remediation_services,arts_entertainment_and_recreation,accomodation_and_food_services,real_estate_and_rental_and_leasing,professional_scientific_and_technical_services,educational_services,health_care_and_social_assistance,other_services_except_government_and_government_enterprises_,government_and_government_enterprises
0,United States,2.1,0.06,0.11,-0.26,-0.09,0.43,-0.03,0.01,0.46,...,0.16,0.10,0.02,0.17,0.11,0.42,0.08,0.19,0.05,0.01
1,New England,2.1,0.01,0.01,-0.24,-0.09,0.39,-0.02,0.02,0.33,...,0.20,0.12,0.04,0.17,0.09,0.56,0.23,0.21,0.05,-0.01
2,Connecticut,2.1,0.00,0.00,-0.27,-0.15,0.39,-0.01,0.02,0.71,...,0.19,0.11,0.04,0.11,0.03,0.37,0.27,0.17,0.07,-0.04
3,Maine,2.1,0.01,0.00,-0.22,0.04,0.72,0.01,-0.02,0.29,...,0.27,0.15,0.05,0.33,0.07,0.26,0.06,0.15,0.05,-0.10
4,Massachusetts,2.2,0.02,0.01,-0.21,-0.11,0.31,-0.03,0.04,0.21,...,0.20,0.10,0.03,0.16,0.13,0.72,0.24,0.22,0.05,0.00


## 2.0 Employment by states
**States Data**

- NY: https://www.labor.ny.gov/stats/2019-2021-Statewide-Short-Term-Industry-Projections.xlsx


In [225]:
# headers = {'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.132 Safari/537.36"}

# url="https://labor.ny.gov/stats/2016-2026-Statewide-and-Regional-Long-Term-Industry-Projections.xlsx"
# s=requests.get(url, headers= headers).text

### New York

In [265]:
employment_ny = pd.read_excel(os.path.join(os.getcwd(), "data/2019-2021-Statewide-Short-Term-Industry-Projections.xlsx"))

In [266]:
employment_ny.head(10)

,Unnamed: 0,New York State Department of Labor,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,"Short-Term Industry Employment Projections, 20...",NaN,NaN,NaN,NaN,NaN
1,NaN,New York State,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NAICS\nCode,Industry Title,Employment,NaN,Net\nChange,Percent\nChange
4,NaN,NaN,NaN,2019,2021,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,000000,Total All Industries,10234640,10435820,201180,0.0196568
7,NaN,NaN,NaN,,,NaN,NaN
8,NaN,00601,"Total Self Employed, All Jobs",570590,577460,6870,0.0120402
9,NaN,NaN,NaN,,,NaN,NaN


In [267]:
employment_ny = employment_ny.dropna(axis=1,how='all')

In [268]:
employment_ny

,New York State Department of Labor,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,"Short-Term Industry Employment Projections, 20...",NaN,NaN,NaN,NaN,NaN
1,New York State,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NAICS\nCode,Industry Title,Employment,NaN,Net\nChange,Percent\nChange
4,NaN,NaN,2019,2021,NaN,NaN
...,...,...,...,...,...,...
149,NaN,State Office Campus Bldg. 12,NaN,NaN,NaN,NaN
150,NaN,Room 490,NaN,NaN,NaN,NaN
151,NaN,"Albany, NY 12240",NaN,NaN,NaN,NaN
152,NaN,Phone: (518) 457-3800,NaN,NaN,NaN,NaN


In [272]:
employment_ny = employment_ny.dropna(axis=0, how="any").reset_index(drop = True)

In [274]:
employment_ny.shape

(109, 6)

In [275]:
employment_ny.columns = ["naics_code","industry","2019","2021","net_change","pct_change"]

- `00601`: Self Employed Workers, All Jobs
- `11`: Agriculture, Forestry, Fishing and Hunting
- `21`: Mining
- `22`: Utilities
- `23`: Construction
- `31`: Manufacturing
- `42`: Wholesale Trade
- `44`: Retail Trade
- `48`: Transportation and Warehousing
- `51`: Information
- `1023`: Financial Activities
- `1024`: Professional and Business Services
- `61`: Educational Services
- `62`: Health Care and Social Assistance
- `71`: Arts, Entertainment, and Recreation
- `72`: Accommodation and Food Services
- `81`: Other Services (except Government)
- `9`: Government

In [276]:
employment_ny.naics_code = employment_ny.naics_code.astype(int)
employment_ny['2019'] = employment_ny['2019'].astype(int)
employment_ny['2021'] = employment_ny['2021'].astype(int)

In [277]:
industry_codes = [601,11,21,22,23,31,42,44,48,51,1023,1024,61,62,71,72,81,9,]

In [278]:
employment_ny_inds = employment_ny[employment_ny.naics_code.isin(industry_codes)]

In [279]:
employment_ny_inds

,naics_code,industry,2016,2026,net_change,pct_change
1,601,"Total Self Employed, All Jobs",570590,577460,6870,0.0120402
2,11,"Agriculture, Forestry, Fishing and Hunting",31180,31870,690,0.0221296
8,21,Mining,4230,4550,320,0.0756501
12,22,Utilities,36700,36430,-270,-0.00735695
13,23,Construction,380940,390570,9630,0.0252796
17,31,Manufacturing,441060,439460,-1600,-0.00362762
39,42,Wholesale Trade,322100,318890,-3210,-0.00996585
43,44,Retail Trade,930340,913410,-16930,-0.0181976
56,48,Transportation and Warehousing,279600,289240,9640,0.0344778
67,51,Information,273110,276340,3230,0.0118267


### New Jersey
- data: https://nj.gov/labor/lpa/employ/indoccpj/indoccpj_index.html
- 2016 - 2019

In [3]:
nj_dir = os.path.join(os.getcwd(),"data/nj_employment.xlsx")

In [15]:
employment_detailed_nj = pd.read_excel(nj_dir,sheet_name=1)

In [16]:
employment_detailed_nj = employment_detailed_nj.dropna(axis=0, how="all").dropna(axis=0,how="all")

In [38]:
employment_nj = pd.read_excel(nj_dir,sheet_name=0).dropna(axis=0, how="all").dropna(axis=0,how="all").loc[4:22]

In [39]:
employment_nj.columns = ["industry","2016","2026","net_change","pct_change"]

In [45]:
def get_nj_employment():
    nj_dir = os.path.join(os.getcwd(),"data/nj_employment.xlsx")
    employment_nj = pd.read_excel(nj_dir,sheet_name=0).dropna(axis=0, how="all").dropna(axis=0,how="all").loc[4:22]
    employment_nj.columns = ["industry","2016","2026","net_change","pct_change"]
    employment_nj = employment_nj.reset_index(drop = True)
    return employment_nj

In [46]:
get_nj_employment()

,industry,2016,2026,net_change,pct_change
0,Total Nonfarm,4073600,4402180,328580,0.778744
1,Total Private Sector,3792200,4134190,341990,0.867189
2,Goods Producing,397000,429740,32740,0.795589
3,Natural Resources and Mining,1300,1400,100,0.743833
4,Construction,153600,176260,22660,1.3856
5,Manufacturing,242100,252080,9980,0.404773
6,Services Providing,3676600,3972450,295850,0.776949
7,"Trade, Transportation, and Utilities",868000,916080,48080,0.540576
8,Information,71500,67950,-3550,-0.507959
9,Financial Activities,244300,251390,7090,0.286495


### Colorado
- data: https://data.colorado.gov/api/views/u2t6-bfhr/rows.csv?accessType=DOWNLOAD

In [384]:
url = "https://data.colorado.gov/api/views/u2t6-bfhr/rows.csv?accessType=DOWNLOAD"

In [385]:
employment_co = pd.read_csv(url)

In [386]:
employment_co["perioddesc"].unique()

array(['Colorado Short-Term Projections, 2016-2018, June 2016',
       'Colorado Short-Term Projections, 2017-2019, August 2017'],
      dtype=object)

In [387]:
employment_co = employment_co[employment_co.areatyname == "State"]

In [388]:
employment_co["perioddesc"].unique()

array(['Colorado Short-Term Projections, 2016-2018, June 2016',
       'Colorado Short-Term Projections, 2017-2019, August 2017'],
      dtype=object)

In [389]:
employment_co = employment_co[employment_co["perioddesc"] == "Colorado Short-Term Projections, 2017-2019, August 2017"]

In [391]:
employment_co.columns

Index(['stateabbrv', 'statename', 'stfips', 'areatype', 'areatyname', 'area',
       'areaname', 'periodid', 'perioddesc', 'periodtype', 'matincodty',
       'matincode', 'matintitle', 'matoccodty', 'matoccode', 'matocctitl',
       'estemp', 'projemp', 'pctestind', 'pctestocc', 'pctprojind',
       'pctprojocc', 'nchg', 'pchg', 'growrate', 'aopeng', 'aopenr', 'aopent',
       'suppress'],
      dtype='object')

In [401]:
employment_co[['matincodty','matincode', 'matintitle', 'matoccodty','estemp']].drop_duplicates()

,matincodty,matincode,matintitle,matoccodty,estemp
74598,2,0,Total All Industries,2,2864028
74599,2,67,"Self Employed and Unpaid Family Workers, All Jobs",2,187410
74600,2,670,"Self Employed and Unpaid Family Workers, All Jobs",2,187410
84241,2,622000,Hospitals,2,68
84562,2,102800,Government,2,19
...,...,...,...,...,...
112051,2,102100,"Trade, Transportation, and Utilities",2,42
112052,2,102800,Government,2,29
112053,2,910000,Total Federal Government Employment,2,29
112054,2,900000,Government,2,29


## California
- data: https://www.labormarketinfo.edd.ca.gov/file/indproj/CA$IndProj2019-2021.xlsx
- 2nd quarter

In [378]:
ca_dir = os.path.join(os.getcwd(),"data/ca_employment.xlsx")
employment_ca = pd.read_excel(ca_dir,sheet_name=0).dropna(axis=0, how="all").dropna(axis=0,how="all")

In [406]:
employment_ca = employment_ca.loc[4:273]

In [408]:
employment_ca.columns = ['naics_1',"naics_code_2","industry","2019","2021","net_change","pct_change"]

In [416]:
employment_ca.head()

,naics_1,naics_code_2,industry,2019,2021,net_change,pct_change
4,6010.0,NaN,Self Employment (A),1201700,1235300,33600,0.0279604
5,8010.0,NaN,Private Household Workers (B),36800,39200,2400,0.0652174
6,110000.0,NaN,Total Farm,468800,482300,13500,0.0287969
7,0.0,NaN,Total Nonfarm,17454400,17955600,501200,0.0287148
8,101100.0,"1133,21",Mining and Logging,23000,23600,600,0.026087


In [410]:
def get_ca_employment():
    ca_dir = os.path.join(os.getcwd(),"data/ca_employment.xlsx")
    employment_ca = pd.read_excel(ca_dir,sheet_name=0).dropna(axis=0, how="all").dropna(axis=0,how="all").loc[4:273]
    employment_ca.columns = ['naics_1',"naics_code_2","industry","2019","2021","net_change","pct_change"]
    employment_ca = employment_ca.reset_index(drop = True)
    return employment_ca

In [415]:
get_ca_employment().head()

,naics_1,naics_code_2,industry,2019,2021,net_change,pct_change
0,6010.0,NaN,Self Employment (A),1201700,1235300,33600,0.0279604
1,8010.0,NaN,Private Household Workers (B),36800,39200,2400,0.0652174
2,110000.0,NaN,Total Farm,468800,482300,13500,0.0287969
3,0.0,NaN,Total Nonfarm,17454400,17955600,501200,0.0287148
4,101100.0,"1133,21",Mining and Logging,23000,23600,600,0.026087


## Pennsylvania


In [437]:
pa_dir = os.path.join(os.getcwd(),"data/pa_employment.xlsx")
employment_pa = pd.read_excel(pa_dir,sheet_name=0).dropna(axis=0, how="all").dropna(axis=0,how="all")

In [438]:
employment_pa = employment_pa.loc[5:120].drop(columns = "Unnamed: 6")

In [439]:
employment_pa.columns = ["naics_code","industry","2019","2021","net_change","pct_change"]

In [440]:
employment_pa

,naics_code,industry,2019,2021,net_change,pct_change
5,NaN,TOTAL JOBS,6407280,6508920.0,101640,0.016
7,NaN,GOODS-PRODUCING,933440,950460.0,17020,0.018
8,11,"Agriculture, Forestry, Fishing & Hunting",61820,61910.0,90,0.001
9,111,Crop Production,35310,35340.0,30,0.001
10,112,Animal Production,22280,22280.0,0,0
...,...,...,...,...,...,...
116,99,Government,371150,371060.0,-90,0
117,491,Postal Service,27640,27020.0,-620,-0.022
118,991,"Federal, Except USPS",69880,70020.0,140,0.002
119,992,"State Gov't, Except Education & Hospitals",90770,90050.0,-720,-0.008


## Short Term Occupational Projections

In [49]:
data_dir = os.path.join(os.getcwd(),"data/ShortTermOccupationalProjections.csv")

In [51]:
occupational_proj_df = pd.read_csv(data_dir)

In [52]:
occupational_proj_df.shape

(32393, 11)

In [53]:
occupational_proj_df.head()

,STFIPS,AreaName,OccupationCode,OccupationName,BaseYear,Base,ProjYear,Proj,Change,PercentChange,AvgAnnualOpenings
0,1,Alabama,"=""13-2011""",Accountants and Auditors,2019,17780,2021,18550,770,4.3,2080
1,2,Alaska,"=""13-2011""",Accountants and Auditors,2019,1940,2021,1980,40,2.1,200
2,4,Arizona,"=""13-2011""",Accountants and Auditors,2019,22930,2021,24400,1470,6.4,2940
3,5,Arkansas,"=""13-2011""",Accountants and Auditors,2019,9020,2021,9250,230,2.5,970
4,6,California,"=""13-2011""",Accountants and Auditors,2019,167400,2021,172300,4900,2.9,18270


## Industry and Occupation Code list
### Occupation Code

In [198]:
data_dir = os.path.join(os.getcwd(),"data/industry_occupa_code_list.xlsx")

In [199]:
occ_code_df = pd.read_excel(data_dir,sheet_name=1)

In [200]:
occ_code_df = occ_code_df.dropna(axis=0, how = "all").dropna(axis = 1, how = "all")

In [201]:
occ_code_df = occ_code_df.loc[5:665]

In [202]:
temp = occ_code_df['Unnamed: 2'].str.contains("-").to_frame().reset_index()

In [203]:
temp_index = temp.loc[temp['Unnamed: 2'] == True,"index"].values

In [204]:
occ_code_cat_df = occ_code_df.loc[temp_index]

In [205]:
occ_code_detail_df = occ_code_df.drop(index=temp_index)

#### wrangling code details df

In [206]:
occ_code_detail_df.shape

(571, 4)

In [207]:
occ_code_detail_df.isnull().sum()

Table A2. 2018 Census Occupation Code List       571
Unnamed: 1                                         0
Unnamed: 2                                         0
Unnamed: 3                                         0
dtype: int64

In [208]:
occ_code_detail_df = occ_code_detail_df.dropna(axis=1,how="all")

In [209]:
occ_code_detail_df.columns = ["census_title", "census_code","soc_code"]

In [210]:
occ_code_detail_df = occ_code_detail_df.iloc[1:]

In [213]:
occ_code_detail_df.head()

,census_title,census_code,soc_code
13,Chief executives,0010,11-1011
14,General and operations managers,0020,11-1021
15,Legislators,0030,11-1031
16,Advertising and promotions managers,0040,11-2011
17,Marketing managers,0051,11-2021


#### wrangling code cat df

In [215]:
occ_code_cat_df.columns = ["temp","census_title","census_code","soc_code"]

In [221]:
mask = occ_code_cat_df.census_title.isna()
occ_code_cat_df.loc[mask, 'census_title'] = occ_code_cat_df.loc[mask, 'temp']

In [223]:
occ_code_cat_df = occ_code_cat_df.drop(columns="temp")

In [226]:
occ_code_cat_df["census_title"] = occ_code_cat_df.census_title.str.replace(":","")

In [229]:
occ_code_cat_df.head()

,census_title,census_code,soc_code
7,"Management, Business, Science, and Arts Occup...",0010-3550,11-0000 - 29-0000
9,"Management, Business, and Financial Occupations",0010-0960,11-0000 - 13-0000
11,Management Occupations,0010-0440,11-0000
47,Business and Financial Operations Occupations,0500-0960,13-0000
79,"Computer, Engineering, and Science Occupations",1005-1980,15-0000 - 19-0000


### Industry Code

In [230]:
inds_code_df = pd.read_excel(data_dir,sheet_name=0)

In [235]:
inds_code_df = inds_code_df.dropna(axis=0, how="all").dropna(axis=1, how="all")

In [238]:
inds_code_df.columns = ["temp","industry_title","census_code","naics_code"]

In [245]:
inds_code_df = inds_code_df.loc[7:353]

In [246]:
inds_code_df.head()

,temp,industry_title,census_code,naics_code
7,"Agriculture, Forestry, Fishing, and Hunting, a...",NaN,0170-0490,11-21
9,NaN,"Agriculture, Forestry, Fishing, and Hunting",0170-0290,11
11,NaN,Crop production,0170,111
12,NaN,Animal production and aquaculture,0180,112
13,NaN,Forestry except logging,0190,"1131, 1132"


In [251]:
temp = inds_code_df['census_code'].str.contains("-").to_frame().reset_index()

In [258]:
temp_index = temp.loc[temp['census_code'] == True,"index"].values

In [260]:
inds_cat_code_df = inds_code_df.loc[temp_index]

In [263]:
mask = inds_cat_code_df["industry_title"].isna()

In [265]:
inds_cat_code_df.loc[mask,"industry_title"] = inds_cat_code_df.loc[mask,"temp"]

In [270]:
inds_cat_code_df.iloc[0]["temp"]

'Agriculture, Forestry, Fishing, and Hunting, and Mining'

In [289]:
idx_remove = [7,175,216,235,266,294]

In [292]:
inds_cat_code_df = inds_cat_code_df.drop(index=idx_remove,columns="temp")

In [294]:
inds_cat_code_df[["census_code_start","census_code_end"]] = inds_cat_code_df["census_code"].str.split("-",expand=True)

In [314]:
inds_cat_code_df = inds_cat_code_df.drop(columns="census_code")

### Join Industry code and Occ code

In [315]:
occ_code_detail_df["census_code"] = occ_code_detail_df.census_code.astype(int)

In [316]:
inds_cat_code_df["census_code_start"] = inds_cat_code_df.census_code_start.astype(int)

In [317]:
inds_cat_code_df["census_code_end"] = inds_cat_code_df.census_code_end.astype(int)

In [318]:
a = occ_code_detail_df.census_code.values
bh = inds_cat_code_df.census_code_end.values
bl = inds_cat_code_df.census_code_start.values

i, j = np.where((a[:, None] >= bl) & (a[:, None] <= bh))

In [319]:
joined = pd.DataFrame(
            np.column_stack([occ_code_detail_df.values[i], inds_cat_code_df.values[j]]),
            columns=occ_code_detail_df.columns.append(inds_cat_code_df.columns))

In [320]:
joined.head()

,census_title,census_code,soc_code,industry_title,naics_code,census_code_start,census_code_end
0,"Farmers, ranchers, and other agricultural mana...",205,11-9013,"Agriculture, Forestry, Fishing, and Hunting",11,170,290
1,Construction managers,220,11-9021,"Agriculture, Forestry, Fishing, and Hunting",11,170,290
2,Education and childcare administrators,230,11-9030,"Agriculture, Forestry, Fishing, and Hunting",11,170,290
3,Postmasters and mail superintendents,400,11-9131,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
4,"Property, real estate, and community associati...",410,11-9141,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490


In [322]:
joined.head(20)

,census_title,census_code,soc_code,industry_title,naics_code,census_code_start,census_code_end
0,"Farmers, ranchers, and other agricultural mana...",205,11-9013,"Agriculture, Forestry, Fishing, and Hunting",11,170,290
1,Construction managers,220,11-9021,"Agriculture, Forestry, Fishing, and Hunting",11,170,290
2,Education and childcare administrators,230,11-9030,"Agriculture, Forestry, Fishing, and Hunting",11,170,290
3,Postmasters and mail superintendents,400,11-9131,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
4,"Property, real estate, and community associati...",410,11-9141,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
5,Social and community service managers,420,11-9151,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
6,Emergency management directors,425,11-9161,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
7,"Personal service managers, all other",426,11-9179,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
8,"Managers, all other",440,11-9199,"Mining, Quarrying, and Oil and Gas Extraction",21,370,490
9,Cost estimators,600,13-1051,Utilities,22,570,690


## 3. Covid infection data
- data is directly from John Hopkins github repo: https://github.com/CSSEGISandData/COVID-19


In [313]:
def get_covid_data(date):
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{}.csv".format(date)
    covid19_df = pd.read_csv(url, index_col=0)
    covid19_df = covid19_df.reset_index()
    covid19_df = covid19_df[covid19_df["Country_Region"] == "US"]
    return covid19_df

In [314]:
covid19_df = get_covid_data(date="03-28-2020")

In [316]:
covid19_df.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,45001.0,Abbeville,South Carolina,US,2020-03-28 23:05:37,34.223334,-82.461707,3,0,0,0,"Abbeville, South Carolina, US"
1,22001.0,Acadia,Louisiana,US,2020-03-28 23:05:37,30.295065,-92.414197,9,1,0,0,"Acadia, Louisiana, US"
2,51001.0,Accomack,Virginia,US,2020-03-28 23:05:37,37.767072,-75.632346,2,0,0,0,"Accomack, Virginia, US"
3,16001.0,Ada,Idaho,US,2020-03-28 23:05:37,43.452658,-116.241552,76,0,0,0,"Ada, Idaho, US"
4,19001.0,Adair,Iowa,US,2020-03-28 23:05:37,41.330756,-94.471059,1,0,0,0,"Adair, Iowa, US"


## 4.0 Populations by States 2019

- link to census data: 
https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv


In [319]:
def get_population_data():
    url = "https://www2.census.gov/programs-surveys/popest/datasets/2010-2019/state/detail/SCPRC-EST2019-18+POP-RES.csv"
    df = pd.read_csv(url)
    return df

In [320]:
pop_2019 = get_population_data()

In [322]:
pop_2019.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,POPESTIMATE2019,POPEST18PLUS2019,PCNT_POPEST18PLUS
0,10,0,0,0,United States,328239523,255200373,77.7
1,40,3,6,1,Alabama,4903185,3814879,77.8
2,40,4,9,2,Alaska,731545,551562,75.4
3,40,4,8,4,Arizona,7278717,5638481,77.5
4,40,3,7,5,Arkansas,3017804,2317649,76.8


In [324]:
pop_2019.NAME.unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming',
       'Puerto Rico Commonwealth'], dtype=object)